# Online Evaluation

We pretrain a PMF model as the environment simulator, i.e., to predict an item's feedback that the user never rates before. The online evaluation procedure follows the Training Algorithm, i.e., the parameters continuously update during the online evaluation stage. Its major difference is that the feedback of a recommended item is observed by the environment simulator. 

In [ ]:
import sys 
sys.path.append('..')

#Dependencies
import os
import json 
import yaml
import pickle
from random import sample
from tqdm import tqdm

import pandas as pd
import numpy as np

from src.environment import OfflineEnv, OfflineFairEnv
from src.model.recommender import DRRAgent, FairRecAgent

from src.recsys_fair_metrics.recsys_fair import RecsysFair

from IPython.display import clear_output
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import recmetrics as rm

py.init_notebook_mode(connected=True)


ENV = dict(drr=OfflineEnv, fairrec=OfflineFairEnv)
AGENT = dict(drr=DRRAgent, fairrec=FairRecAgent)

In [ ]:
dataset_name = "movie_lens_100k"
dataset_path = "../data/{}_output_path.json".format(dataset_name)
with open(dataset_path) as json_file:
    _dataset_path = json.load(json_file)


dataset = {}
with open(os.path.join("..", _dataset_path["eval_users_dict"]), "rb") as pkl_file:
    dataset["eval_users_dict"] = pickle.load(pkl_file)

with open(os.path.join("..", _dataset_path["eval_users_history_lens"]), "rb") as pkl_file:
    dataset["eval_users_history_lens"] = pickle.load(pkl_file)

with open(os.path.join("..", _dataset_path["users_history_lens"]), "rb") as pkl_file:
    dataset["users_history_lens"] = pickle.load(pkl_file)

with open(os.path.join("..", _dataset_path["item_groups"]), "rb") as pkl_file:
    dataset["item_groups"] = pickle.load(pkl_file)

item_groups_df = pd.DataFrame(dataset["item_groups"].items(), columns=["item_id", "group"])
catalog = item_groups_df.item_id.unique().tolist()

# Actor-Critic Models

In [ ]:
drr_train_ids = [
    "movie_lens_100k_2022-03-23_16-59-47"
]

fairrec_train_ids = [   
    "movie_lens_100k_fair_2022-03-23_16-59-43"
]

idx = -1


algorithm = "fairrec"
train_version = dataset_name if algorithm == "drr" else "{}_fair".format(dataset_name)
train_id = drr_train_ids[idx] if algorithm == "drr" else fairrec_train_ids[idx]
output_path = "../model/{}/{}".format(train_version, train_id)

path = os.path.abspath(
    os.path.join(output_path, "{}.yaml".format(train_version))
)
with open(path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)


no_cuda = False
top_k = None


In [ ]:
ratings = pd.read_csv(_dataset_path["ratings_df"])

fig = plt.figure(figsize=(15, 7))
fig.patch.set_facecolor("white")
rm.long_tail_plot(
    df=ratings, 
    item_id_column="movie_id", 
    interaction_type="items clicked", 
    percentage=0.5,
    x_labels=False
)
plt.savefig(os.path.join(output_path, "long_tail_plot.png"))

In [ ]:
_precision = []
_propfair = []
_ufg = []
for i in range(1):
    sum_precision = 0
    sum_propfair = 0
    sum_reward = 0

    recommended_item = []
    random_recommended_item = []

    env = ENV[algorithm](
        users_dict=dataset["eval_users_dict"],
        users_history_lens=dataset["eval_users_history_lens"],
        n_groups=config["model_train"]["n_groups"],
        item_groups=dataset["item_groups"],
        state_size=config["model_train"]["state_size"],
        done_count=config["model_train"]["done_count"],
        fairness_constraints=config["model_train"]["fairness_constraints"],
        reward_threshold=config["model_train"]["reward_threshold"],
        use_only_reward_model=True,
    )
    available_users = env.available_users

    recommender = AGENT[algorithm](
        env=env,
        train_version=train_version,
        is_test=True,
        model_path=output_path,
        users_num=config["model_train"]["users_num"],
        items_num=config["model_train"]["items_num"],
        embedding_dim=config["model_train"]["embedding_dim"],
        srm_size=config["model_train"]["srm_size"],
        state_size=config["model_train"]["state_size"],
        actor_hidden_dim=config["model_train"]["actor_hidden_dim"],
        actor_learning_rate=config["model_train"]["actor_learning_rate"],
        critic_hidden_dim=config["model_train"]["critic_hidden_dim"],
        critic_learning_rate=config["model_train"]["critic_learning_rate"],
        discount_factor=config["model_train"]["discount_factor"],
        tau=config["model_train"]["tau"],
        learning_starts=1,#config["model_train"]["learning_starts"],
        replay_memory_size=config["model_train"]["replay_memory_size"],
        batch_size=1,#config["model_train"]["batch_size"],
        embedding_network_weights_path="../{}".format(config["model_train"]["embedding_network_weights"]),
        n_groups=config["model_train"]["n_groups"],
        fairness_constraints=config["model_train"]["fairness_constraints"],
        use_reward_model=config["model_train"]["use_reward_model"],
    )

    for user_id in tqdm(available_users):

        eval_env = ENV[algorithm](
            users_dict=dataset["eval_users_dict"],
            users_history_lens=dataset["eval_users_history_lens"],
            n_groups=config["model_train"]["n_groups"],
            item_groups=dataset["item_groups"],
            state_size=config["model_train"]["state_size"],
            done_count=config["model_train"]["done_count"],
            fairness_constraints=config["model_train"]["fairness_constraints"],
            reward_threshold=config["model_train"]["reward_threshold"],
            fix_user_id=user_id,
            reward_model=recommender.reward_model,
            device=recommender.device,
            use_only_reward_model=True,
        )

        recommender.env = eval_env

        # recommender.buffer = pickle.load(open(os.path.join(output_path, "buffer.pkl"), "rb"))

        precision, ndcg, propfair, reward, list_recommended_item = recommender.train(
            max_episode_num=1, top_k=top_k, load_model=True
        )
        recommended_item.append(list_recommended_item)
        random_recommended_item.append({user_id: sample(catalog, config["model_train"]["done_count"])})

        sum_precision += precision
        sum_propfair += propfair
        sum_reward += reward

        del eval_env


    _precision.append(sum_precision / len(dataset["eval_users_dict"]))
    _propfair.append(sum_propfair / len(dataset["eval_users_dict"]))
    _ufg.append((sum_propfair / len(dataset["eval_users_dict"]))
        / (1 - (sum_precision / len(dataset["eval_users_dict"]))))


clear_output(wait=True)

print("PropFair ", round(np.mean(_propfair), 4))
print("Precision ", round(np.mean(_precision), 4))
print("UFG ", round(np.mean(_ufg), 4))

# RecMetrics

In [ ]:
fig = go.Figure([
    go.Bar(
        x=["precision", "prop_fair", "ufg"],
        y=[round(np.mean(_precision), 4), round(np.mean(_propfair), 4), round(np.mean(_ufg), 4)],
        text=[round(np.mean(_precision), 4), round(np.mean(_propfair), 4), round(np.mean(_ufg), 4)],
        textposition="auto",
    ),
])


fig.update_layout(
    title="Fair Metrics",
    template="ggplot2",
    width=1000, 
    font=dict(size=16)
)
fig.show()
fig.write_image(os.path.join(output_path, "metrics.png"))

In [ ]:
recs = pd.DataFrame([i.values() for i in recommended_item], columns=["sorted_actions"]).sorted_actions.values.tolist()
random_recs = pd.DataFrame([i.values() for i in random_recommended_item], columns=["sorted_actions"]).sorted_actions.values.tolist()

In [ ]:
coverage = rm.prediction_coverage(recs, catalog)
random_coverage = rm.prediction_coverage(random_recs, catalog)
fig = go.Figure([
    go.Bar(
        x=[train_version, "random"],
        y=[coverage, random_coverage],
        text=[coverage, random_coverage],
        textposition="auto",
    ),
])


fig.update_layout(
    title="Catalog Coverage in %",
    template="ggplot2",
    yaxis_title="coverage",
    yaxis = dict(
        ticksuffix="%",
    ),
    width=1000, 
    font=dict(size=16)
)
fig.show()
fig.write_image(os.path.join(output_path, "coverage.png"))

In [ ]:
personalization = round(rm.personalization(recs) * 100, 2)
random_personalization = round(rm.personalization(random_recs) * 100, 2)
fig = go.Figure([
    go.Bar(
        x=[train_version, "random"],
        y=[personalization, random_personalization],
        text=[personalization, random_personalization],
        textposition="auto",
    ),
])


fig.update_layout(
    title="Personalization",
    template="ggplot2",
    yaxis_title="personalization",
    yaxis = dict(
        ticksuffix = "%",
    ),
    width=1000, 
    font=dict(size=16)
)
fig.show()
fig.write_image(os.path.join(output_path, "personalization.png"))

In [ ]:
feature_df = pd.DataFrame(item_groups_df[["item_id"]].apply(lambda x: recommender.get_items_emb(x).cpu().numpy().tolist())["item_id"].tolist())

intra_list_similarity = rm.intra_list_similarity(recs, feature_df)
random_intra_list_similarity = rm.intra_list_similarity(random_recs, feature_df)

fig = go.Figure([
    go.Bar(
        x=[train_version, "random"],
        y=[intra_list_similarity, random_intra_list_similarity],
        text=[intra_list_similarity, random_intra_list_similarity],
        textposition="auto",
    ),
])


fig.update_layout(
    title="Intra-list Similarity",
    template="ggplot2",
    yaxis_title="similarity",
    # yaxis = dict(
    #     ticksuffix = "%",
    # ),
    width=1000, 
    font=dict(size=16)
)
fig.show()
fig.write_image(os.path.join(output_path, "intra_list_similarity.png"))

# Exposure

In [ ]:
_df = pd.DataFrame([i.values() for i in recommended_item], columns=["sorted_actions"])
_df["user_id"] = [list(i.keys())[0] for i in recommended_item]
_item_metadata = pd.DataFrame(dataset["item_groups"].items(), columns=["item_id", "group"])

user_column = "user_id"
item_column = "item_id"
reclist_column = "sorted_actions"

recsys_fair = RecsysFair(
    df = _df, 
    supp_metadata = _item_metadata,
    user_column = user_column, 
    item_column = item_column, 
    reclist_column = reclist_column, 
)

fair_column = "group"
ex = recsys_fair.exposure(fair_column, config["model_train"]["done_count"])

In [ ]:
fig = ex.show(kind='per_group_norm', column=fair_column)
fig.show()
fig.write_image(os.path.join(output_path, "exposure_per_group.png"))

In [ ]:
fig = ex.show(kind='per_rank_pos', column=fair_column)
fig.show()
fig.write_image(os.path.join(output_path, "exposure_per_rank.png"))